### This example applies DeepPhasePick on seismic data downloaded using FDSN web service client for ObsPy.

In [1]:
import os
os.chdir('../')
import config, data, model, util                                                
from obspy.clients.fdsn import Client                                           
import obspy.core as oc 

## 1. Configure DPP

In [2]:
# config
util.init_session()
dpp_config = config.Config()
dpp_config.set_trigger(pthres_p=[0.9, 0.001], pthres_s=[0.9, 0.001])
dpp_config.set_picking(mcd_iter=10, run_mcd=True)
# dpp_config.set_picking(run_mcd=False)
#
dpp_config.set_data(
    stas=['PB01', 'PB02'],
    net='CX',
    ch='HH',
    archive='sample_data/CX_20140401',
    opath='out_CX_20140401'
)
dpp_config.set_time(
    dt_iter=3600.,
    tstart = "2014-04-01T02:00:00",
    tend = "2014-04-01T03:00:00",    
)

__pycache__/ removed
~/.nv/ not found, continuing...


## 2. Download seismic data and read it into DPP

In [3]:
# download and archive seismic waveforms
client = Client("GFZ")
os.makedirs(f"{dpp_config.data['archive']}", exist_ok=True)
tstart = oc.UTCDateTime(dpp_config.time['tstart'])
tend = oc.UTCDateTime(dpp_config.time['tend'])
#
for sta in dpp_config.data['stas']:
    st = client.get_waveforms(network=dpp_config.data['net'], station=sta, location="*", channel=f"{dpp_config.data['ch']}?", starttime=tstart, endtime=tend)
    # print(st)
    st_name = f"{dpp_config.data['archive']}/{st[0].stats.network}.{st[0].stats.station}..{st[0].stats.channel[:-1]}.mseed"
    print(f"writing stream {st_name}...")
    st.write(st_name, format="MSEED")

writing stream sample_data/CX_20140401/CX.PB01..HH.mseed...
writing stream sample_data/CX_20140401/CX.PB02..HH.mseed...


In [4]:
# data
dpp_data = data.Data()
dpp_data.read_from_directory(dpp_config)
#
# for k in dpp_data.data:
#     print(k, dpp_data.data[k])

reading seismic waveform: sample_data/CX_20140401/CX.PB01..HH.mseed
reading seismic waveform: sample_data/CX_20140401/CX.PB02..HH.mseed
6 Trace(s) in Stream:
CX.PB01..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHE | 2014-04-01T01:59:59.998394Z - 2014-04-01T02:59:59.998394Z | 100.0 Hz, 360001 samples
CX.PB01..HHN | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHE | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHN | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
#
processing raw stream data...
detrend...
resampling...
CX.PB01..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples --> skipped, already sampled at 100.0 Hz
CX.PB01..HHE | 2014-04-01T01:59:59.998394Z -

## 3. Run phase detection and picking

In [5]:
# model
# dpp_model = model.Model(verbose=False)
dpp_model = model.Model(verbose=False, version_pick_P="20201002_2", ntrials_P=50, version_pick_S="20201002_2", ntrials_S=50)
#
# print(dpp_model.model_detection['best_model'].summary())
# print(dpp_model.model_picking_P['best_model'].summary())
# print(dpp_model.model_picking_S['best_model'].summary())

In [6]:
# run phase detection
dpp_model.run_detection(dpp_config, dpp_data, save_dets=False, save_data=False)

#
Calculating predictions for stream: CX.PB01..HH?...
strimming stream: 1, 1
720/720 [==============================] - 28s 39ms/step
3 Trace(s) in Stream:
CX.PB01..HHE | 2014-04-01T01:59:59.998394Z - 2014-04-01T02:59:59.998394Z | 100.0 Hz, 360001 samples
CX.PB01..HHN | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB01..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 13, s_picks = 8
#
Calculating predictions for stream: CX.PB02..HH?...
720/720 [==============================] - 34s 48ms/step
3 Trace(s) in Stream:
CX.PB02..HHE | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHN | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
CX.PB02..HHZ | 2014-04-01T01:59:59.998393Z - 2014-04-01T02:59:59.998393Z | 100.0 Hz, 360001 samples
p_picks = 10, s_picks = 4


In [7]:
# run phase picking
dpp_model.run_picking(dpp_config, dpp_data, save_plots=True, save_stats=True, save_picks=False)

#
1, 2014-04-01T02:00:00.000000Z, 2014-04-01T03:00:00.000000Z, PB01
triggered picks (P, S): 13, 8
selected picks (P, S): 10, 6
#
P pick: 1/10


100%|██████████| 10/10 [00:03<00:00,  2.97it/s]


3.36 3.47 3.41 3.77 out_CX_20140401/20140401T020000_20140401T030000 PB01
plotting predicted phase P: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB01_P_01.png
plotting predicted phase P: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB01_P_mc_01.png
tpred = 3.470
terr(1 x pb_std) = (-0.060, +0.300)
pick_class = 2
pb, pb_std = (0.508, 0.233)
#
P pick: 2/10


100%|██████████| 10/10 [00:01<00:00,  5.11it/s]


3.36 3.62 3.47 3.71 out_CX_20140401/20140401T020000_20140401T030000 PB01
plotting predicted phase P: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB01_P_02.png
plotting predicted phase P: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB01_P_mc_02.png
tpred = 3.620
terr(1 x pb_std) = (-0.150, +0.090)
pick_class = 2
pb, pb_std = (0.502, 0.199)
#
P pick: 3/10


...

2.4 2.19 2.18 2.2 out_CX_20140401/20140401T020000_20140401T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_01.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_01.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_mc_01.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_mc_01.png
tpred = 2.190
terr(1 x pb_std) = (-0.010, +0.010)
pick_class = 0
pb, pb_std = (0.504, 0.071)
#
S pick: 2/3


100%|██████████| 10/10 [00:01<00:00,  7.11it/s]


2.4 2.21 2.18 2.26 out_CX_20140401/20140401T020000_20140401T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_02.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_02.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_mc_02.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_mc_02.png
tpred = 2.210
terr(1 x pb_std) = (-0.030, +0.050)
pick_class = 0
pb, pb_std = (0.511, 0.144)
#
S pick: 3/3


100%|██████████| 10/10 [00:01<00:00,  8.20it/s]


2.4 2.47 2.45 2.49 out_CX_20140401/20140401T020000_20140401T030000 PB02
(480,) (480,)
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_03.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_03.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_E_mc_03.png
plotting predicted phase S: out_CX_20140401/20140401T020000_20140401T030000/pick_plots/PB02_S_N_mc_03.png
tpred = 2.470
terr(1 x pb_std) = (-0.020, +0.020)
pick_class = 0
pb, pb_std = (0.505, 0.135)


## 4. Plot predicted phases

In [8]:
# plots
util.plot_predicted_phases(dpp_config, dpp_data, dpp_model)
# util.plot_predicted_phases(dpp_config, dpp_data, dpp_model, plot_probs=['P','S'], shift_probs=True)

creating plots...
1 PB01 Z 2014-04-01T01:59:59.998393Z 2014-04-01T02:59:59.998393Z
1 PB01 E 2014-04-01T01:59:59.998394Z 2014-04-01T02:59:59.998394Z
1 PB02 Z 2014-04-01T01:59:59.998393Z 2014-04-01T02:59:59.998393Z
1 PB02 E 2014-04-01T01:59:59.998393Z 2014-04-01T02:59:59.998393Z
